In [62]:
import re
import pandas as pd
import numpy as np
from graph_helpers import load_graph_by_name
from collections import defaultdict
from graph_tool.topology import shortest_distance


In [63]:
g = load_graph_by_name('digg')

In [64]:
print(g.num_vertices())
print(g.num_edges())

279631
1548131


In [65]:
node2times = defaultdict(list)
node2occ = defaultdict(list)
with open('logs/test_digg.log', 'r') as f:
    idx = 0
    for l in f:
        if l.startswith('CPP'):
            node, time = re.findall('\{(.*?)\}', l)
            node = int(node)
            time = float(time)
            node2times[node].append(time)
            node2occ[node].append(idx)
            idx += 1
        elif l.startswith('root'):
            print(l)
            idx = 0

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 267154

root is 180561



In [80]:
df = pd.DataFrame(list(zip(node2times.keys(), node2times.values(), node2occ.values())),
                  columns=['node', 'times', 'occ'])

In [81]:
df['mean_time'] = df['times'].apply(lambda r: np.mean(r))
df['std_time'] = df['times'].apply(lambda r: np.std(r))
df['mean_occ'] = df['occ'].apply(lambda r: np.mean(r))

In [82]:
del df['times']
del df['occ']

In [83]:
source = 267154
dist = shortest_distance(g, source=source)

In [84]:
df['dist'] = [dist[n] for n in df['node']]

In [86]:
df.sort_values(by='std_time', ascending=False).head(10)

,node,mean_time,std_time,mean_occ,dist
162,97660,4.579999,2.819693,0.0,3
210,95218,0.004328,0.005191,1.0,3
150,30033,0.002977,0.002874,2.0,3
26,140868,0.001236,0.001350,16.0,2
5,110009,0.000717,0.001284,12.0,2
0,266241,0.000903,0.001099,5.0,3
127,156964,0.000937,0.000980,6.0,3
82,166077,0.000761,0.000924,3.0,2
3,75272,0.000613,0.000840,11.0,3
38,19547,0.000574,0.000805,27.0,3


In [87]:
df[['mean_time', 'mean_occ']].corr(method='spearman')

,mean_time,mean_occ
mean_time,1.000000,-0.624944
mean_occ,-0.624944,1.000000


In [88]:
df[['mean_time', 'dist']].corr(method='spearman')

,mean_time,dist
mean_time,1.000000,0.164057
dist,0.164057,1.000000
